# Into To Depliying App To AWS

Since there is so much to know about AWS, this will mostly focus on a specific use case:
_How to get your app up & running on a server on the internet  so you can test it with friends & family._

We will use the **AWS Free Tier**. We will be creating a new server on AWS, deploying our code, & configuring the server to make it available on the internet.

You will need to sign up for the AWS free tier [here](https://aws.amazon.com/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Types=*all&awsf.Free%20Tier%20Categories=*all).

# Add Key Pair to AWS

TBD

# Create EC2 Server Instance

TBD

# Add Deployment Script & Configs To Project

TBD

# Deploy To Server

TBD

# Update Allowed Hosts & Deploy Changes

TBD